In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from threading import Thread
from time import time
from concurrent.futures import ThreadPoolExecutor
import wikipedia as wp

##### get current constituents and historical changes of SP500 from wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.pandas.read_html(url)
len(tables)

2

In [3]:
tables[0].head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [4]:
tables[1].head()

Date  Added                      Removed  \
                Date Ticker             Security  Ticker   
0      April 2, 2019    DOW             Dow Inc.     BHF   
1  February 27, 2019    WAB   Wabtec Corporation      GT   
2  February 15, 2019    ATO    Atmos Energy Corp     NFX   
3   January 18, 2019    TFX         Teleflex Inc     PCG   
4    January 2, 2019    FRC  First Republic Bank     SCG   

                                       \
                             Security   
0               Brighthouse Financial   
1  The Goodyear Tire & Rubber Company   
2                Newfield Exploration   
3                           PG&E Corp   
4                               SCANA   

                                            Reason  
                                            Reason  
0                        DOW spun off from DWDP[7]  
1      WAB acquiring GE transportation business[8]  
2                             ECA acquiring NFX[9]  
3                    PCG filing for bankruptcy[10]  
4  Dominion Energy acquiring SCANA Corporation[11]

##### get info box on wikipedia

In [5]:
#page = wp.page('IBM')
#url = page.url

url='https://en.wikipedia.org/wiki/IBM'
response = urllib.request.urlopen(url)
text = response.read()
soup = BeautifulSoup(text, 'html.parser')

In [6]:
pd.read_html(url)[0]

,0,1
0,NaN,NaN
1,"IBM headquarters in Armonk, New York.","IBM headquarters in Armonk, New York."
2,Type,Public company
3,Traded as,NYSE: IBMDJIA ComponentS&P 100 ComponentS&P 50...
4,ISIN,US4592001014
5,Industry,Cloud computingComputer hardwareComputer softw...
6,Founded,"June 16, 1911; 107 years agoComputing-Tabulati..."
7,Founders,Charles Ranlett FlintThomas J. Watson Sr.
8,Headquarters,"Armonk, New York, U.S."
9,Area served,177 countries[2]


###### don't use pandas.read_html

you can ues this approach to scrap other things not in a table

In [7]:
# soup is in BeautifulSoup
tr = soup.find("table","infobox vcard").find_all("tr")  # find all needed info in infobox card

In [8]:
len(tr),tr[-1]

(19,
 <tr><th scope="row" style="padding-right:0.5em;">Website</th><td style="line-height:1.35em;"><span class="url"><a class="external text" href="https://www.ibm.com/us-en/" rel="nofollow">www.ibm.com</a></span></td></tr>)

In [9]:
tr[-1].get_text()

'Websitewww.ibm.com'

In [10]:
tr[-1].find("th").get_text(), tr[-1].find("td").get_text()

('Website', 'www.ibm.com')

In [11]:
def get_infobox(name):
    try:
        page = wp.page(name) # get wikipedia search result page
    except:
        return 
    url = page.url
    response = urllib.request.urlopen(url)
    text = response.read()
    soup = BeautifulSoup(text, 'html.parser')
    
    try:
        tr = soup.find("table","infobox vcard").find_all("tr")  # find all needed info in infobox card
        title=[]
        content=[]
        for i in tr[2:]:
            t = i.find("th").get_text()
            c = i.find("td").get_text()
            
            #string parser
            if c[0]=='\n':
                c=c[1:]
            if c[-1]=='\n':
                c=c[:-1]
            c=c.replace('\n',',')           
            t=t.replace('\xa0',' ')
            c=c.replace('\xa0',' ')
            while(c.find('[')!=-1):
                c=c.replace(c[c.find('['):c.find(']')+1],'')
                
            title.append(t)
            content.append(c)
        d = dict(zip(title,content))
    except:
        return 

    return d

In [12]:
d=get_infobox('IBM'); d

{'Type': 'Public company',
 'Traded as': 'NYSE: IBMDJIA ComponentS&P 100 ComponentS&P 500 Component',
 'ISIN': 'US4592001014',
 'Industry': 'Cloud computingComputer hardwareComputer softwareCognitive computing',
 'Founded': 'June 16, 1911; 107 years ago (1911-06-16) (as Computing-Tabulating-Recording Company)Endicott, New York, U.S.',
 'Founders': 'Charles Ranlett FlintThomas J. Watson Sr.',
 'Headquarters': 'Armonk, New York, U.S.',
 'Area served': '177 countries',
 'Key people': 'Ginni Rometty(Chairman, President and CEO)',
 'Products': 'See IBM products',
 'Revenue': ' US$79.59 billion (2018)',
 'Operating income': ' US$13.21 billion (2018)',
 'Net income': ' US$8.72 billion (2018)',
 'Total assets': ' US$123.38 billion (2018)',
 'Total equity': ' US$16.79 billion (2018)',
 'Number of employees': '350,600 (2018)',
 'Website': 'www.ibm.com'}

###### Use multithread

In [13]:
def get_infobox_for_thread(name,d):
    d[name]=get_infobox(name)
     #global n
     #n+=1
     #if not n%100:
     #    print(n)

def get_infobox_for_thread_2(name):
    global dth
    dth[name]=get_infobox(name)

In [14]:
# get_infobox_for_thread

#n=0        
dth={}
threadlist=[]
for i in ['IBM','GOOGLE','Microsoft']:
    t = Thread(target=get_infobox_for_thread,args=(i,dth))
    t.start()
    threadlist.append(t)
for i in threadlist:
    i.join()
dth

{'IBM': {'Type': 'Public company',
  'Traded as': 'NYSE: IBMDJIA ComponentS&P 100 ComponentS&P 500 Component',
  'ISIN': 'US4592001014',
  'Industry': 'Cloud computingComputer hardwareComputer softwareCognitive computing',
  'Founded': 'June 16, 1911; 107 years ago (1911-06-16) (as Computing-Tabulating-Recording Company)Endicott, New York, U.S.',
  'Founders': 'Charles Ranlett FlintThomas J. Watson Sr.',
  'Headquarters': 'Armonk, New York, U.S.',
  'Area served': '177 countries',
  'Key people': 'Ginni Rometty(Chairman, President and CEO)',
  'Products': 'See IBM products',
  'Revenue': ' US$79.59 billion (2018)',
  'Operating income': ' US$13.21 billion (2018)',
  'Net income': ' US$8.72 billion (2018)',
  'Total assets': ' US$123.38 billion (2018)',
  'Total equity': ' US$16.79 billion (2018)',
  'Number of employees': '350,600 (2018)',
  'Website': 'www.ibm.com'},
 'Microsoft': {'Type': 'Public',
  'Traded as': 'NASDAQ: MSFTNASDAQ-100 componentDJIA componentS&P 100 componentS&P 500

In [15]:
# get_infobox_for_thread_2
dth={}
threadlist=[]
for i in ['IBM','GOOGLE','Microsoft']:
    t = Thread(target=get_infobox_for_thread_2,args=(i,))
    t.start()
    threadlist.append(t)
for i in threadlist:
    i.join()
dth

{'IBM': {'Type': 'Public company',
  'Traded as': 'NYSE: IBMDJIA ComponentS&P 100 ComponentS&P 500 Component',
  'ISIN': 'US4592001014',
  'Industry': 'Cloud computingComputer hardwareComputer softwareCognitive computing',
  'Founded': 'June 16, 1911; 107 years ago (1911-06-16) (as Computing-Tabulating-Recording Company)Endicott, New York, U.S.',
  'Founders': 'Charles Ranlett FlintThomas J. Watson Sr.',
  'Headquarters': 'Armonk, New York, U.S.',
  'Area served': '177 countries',
  'Key people': 'Ginni Rometty(Chairman, President and CEO)',
  'Products': 'See IBM products',
  'Revenue': ' US$79.59 billion (2018)',
  'Operating income': ' US$13.21 billion (2018)',
  'Net income': ' US$8.72 billion (2018)',
  'Total assets': ' US$123.38 billion (2018)',
  'Total equity': ' US$16.79 billion (2018)',
  'Number of employees': '350,600 (2018)',
  'Website': 'www.ibm.com'},
 'Microsoft': {'Type': 'Public',
  'Traded as': 'NASDAQ: MSFTNASDAQ-100 componentDJIA componentS&P 100 componentS&P 500

In [16]:
dth = {}
executor = ThreadPoolExecutor(1000)
for i in ['IBM','GOOGLE','Microsoft']:
    executor.submit(get_infobox_for_thread, i,dth)

In [18]:
dth

{'IBM': {'Type': 'Public company',
  'Traded as': 'NYSE: IBMDJIA ComponentS&P 100 ComponentS&P 500 Component',
  'ISIN': 'US4592001014',
  'Industry': 'Cloud computingComputer hardwareComputer softwareCognitive computing',
  'Founded': 'June 16, 1911; 107 years ago (1911-06-16) (as Computing-Tabulating-Recording Company)Endicott, New York, U.S.',
  'Founders': 'Charles Ranlett FlintThomas J. Watson Sr.',
  'Headquarters': 'Armonk, New York, U.S.',
  'Area served': '177 countries',
  'Key people': 'Ginni Rometty(Chairman, President and CEO)',
  'Products': 'See IBM products',
  'Revenue': ' US$79.59 billion (2018)',
  'Operating income': ' US$13.21 billion (2018)',
  'Net income': ' US$8.72 billion (2018)',
  'Total assets': ' US$123.38 billion (2018)',
  'Total equity': ' US$16.79 billion (2018)',
  'Number of employees': '350,600 (2018)',
  'Website': 'www.ibm.com'},
 'Microsoft': {'Type': 'Public',
  'Traded as': 'NASDAQ: MSFTNASDAQ-100 componentDJIA componentS&P 100 componentS&P 500